In [2]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

import time

In [3]:
start = time.time()

stop_words = set(stopwords.words('english'))


train_url = 'proj3_data/split_1/train.tsv'
test_url = 'proj3_data/split_1/test.tsv'
test_y_url = 'proj3_data/split_1/test_y.tsv'
train = pd.read_csv(train_url, sep='\t', header=0, dtype=str)
test =  pd.read_csv(test_url, sep='\t', header=0, dtype=str)
train['review'] = train['review'].str.replace('&lt;.*?&gt;', ' ', regex=True)
test['review'] = test['review'].str.replace('&lt;.*?&gt;', ' ', regex=True)
test_y = pd.read_csv(test_y_url, sep='\t', header=0,dtype=str)
test = test.merge(test_y, on = 'id')

tot = pd.DataFrame()
tot['review'] = pd.concat([train['review'],test['review']])
tot['sentiment'] = pd.concat([train['sentiment'],test['sentiment']])

vectorizer = TfidfVectorizer(
    preprocessor=lambda x: x.lower(),  # Convert to lowercase
    stop_words=stop_words,             # Remove stop words
    ngram_range=(1, 4),               # Use 1- to 4-grams
    min_df=0.001,                        # Minimum term frequency
    max_df=0.5,                       # Maximum document frequency
    token_pattern=r"\b[\w+\|']+\b" # Use word tokenizer: See Ethan's comment below
)

dtm_tot = vectorizer.fit_transform(tot['review'])
feature_names = vectorizer.get_feature_names_out()
dtm_train=dtm_tot[:25000,:]
logreg = LogisticRegression(C=1.01,
                            max_iter=10000, 
                            solver='liblinear',penalty='l1').fit(dtm_train, train['sentiment'])
selected_features = feature_names[logreg.coef_[0]!=0]
vocabulary_dict = {feature: idx for idx, feature in enumerate(selected_features)}

file_path = 'myvocab.txt'

# Write the dictionary to the text file
with open(file_path, 'w') as file:
    for word, index in vocabulary_dict.items():
        file.write(f'{word}: {index}\n')

end = time.time()
end - start

80.1889750957489